# QUBO and Variational Quantum Eigensolvers (VQE)

$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$
$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$
$\newcommand{\expval}[1]{\left\langle{#1}\right\rangle}$
$\DeclareMathOperator*{\argmax}{arg\,max}$
$\DeclareMathOperator*{\argmin}{arg\,min}$
<div style="color:#d3d3d3;text-align:right;">If references are not displayed properly, clear output, restart kernel, and run the following magics.</div>

In [ ]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

In [ ]:
%%javascript
MathJax.Hub.Queue(
  ["resetEquationNumbers", MathJax.InputJax.TeX],
  ["PreProcess", MathJax.Hub],
  ["Reprocess", MathJax.Hub]
);

## QUBO problem definition

QUBO problems, such as the infamous Max-Cut, could generally be defined as follows: 

$$
\begin{equation*}
\min_{x \in \{0,1\}^n} x^T Q x + \mu^T x
\end{equation*}
$$

where $x$ is an $n$-dimensional binary vector, the QUBO matrix $Q$ is an $n \times n$ square matrix, and the QUBO vector $\mu$ is an $n$-dimensional vector. Having $Q$ and $\mu$ for the problem at hand, the objective is to find the binary vector $x$ which minimizes the above expression.

In the case of the Max-Cut problem, $x$ represents the binary coloring scheme of the graph nodes, i.e. $x_i$ would be $0$/$1$ if node $i$ is assigned to set $0$/set $1$. A relevant cost function to be minimized could then be the sum of weights of the edges that connect two nodes from the same set: 

$$
\begin{equation*}
\sum_{i,j=1}^n x_i w_{ij} x_j + \sum_{i,j=1}^n (1-x_i)w_{ij}(1-x_j).
\end{equation*}
$$ 

However, we could also formulate Max-Cut as a maximization problem and then transform it into a minimization problem. It suffices to sum the weights of edges connecting node $i$ from set $1$ to node $j$ from set $0$. This summation should be maximized; however, if we negate this summation, we should minimize. Consequently, the corresponding cost function to be minimized as in a QUBO generic from would be:

$$
\begin{equation*}
C(x) = - \sum_{i,j=1}^n x_i w_{ij} (1 - x_j) = - x^T w x + (wx)^T x
\end{equation*}
$$

in which $-w$ and $wx$ would be the QUBO matrix and the QUBO vector respectively.

## Formulating QUBO as a ground state problem

In quantum mechanics, Hamiltonian ($\hat{H}$) is responsible for the time-evolution of a system. In other words, if you know the Hamiltonian of a system and its initial state, the state of the system could be determined in any other time. One should note that the Hamiltonian of a system could be _time-dependent_. Furthermore, $H$'s might not _commute_ at different times : $[\hat{H}(t_1),\hat{H}(t_2)] \ne 0$.

Here, we are interested in finding the Hamiltonian $\hat{H}_C$ whose eigenstates are $\ket{x}$ with eigenvalues $C(x)$.

$$
\begin{equation*}
\hat{H}_C \ket{x} = C(x) \ket{x}
\end{equation*}
$$

The _ground state_ of this Hamiltonian would then be the bound state with the minimum $C(x)$. In other words, any QUBO problem is equivalent to finding the ground state of a Hamiltonian $\hat{H}_C$. But how to find $\hat{H}_C$ for instance for Max-Cut?

First, we have to select a basis for $\hat{H}_C$ representation. Any set of orthogonal basis vectors in the corresponding Hilbert space could be selected. $Z$-bases with eigenvalues $1$ and $-1$ are convenient for now:

$$
\begin{equation*}
\hat{Z}_i \ket{x} = z_i \ket{x}, \quad z_i=(-1)^{x_i}~~for~~x_i \in \{0,1\}
\end{equation*}
$$

If we represent $C(x)$ in terms of $z_i$'s, the analogous representation of $\hat{H}_C$ in the $Z$-bases would be determined. A possible change of variable for this is: 

$$ 
\begin{equation}
x_i = \frac{1-z_i}{2}.
\label{eq:var_change}
\end{equation}
$$

Consequently, the corresponding Hamiltonian in terms of the Pauli $\hat{Z}$ operators would take the form:

$$
\begin{equation}
\hat{H}_C = - \frac{1}{2} \sum_{i<j} w_{ij} \hat{Z}_i \hat{Z}_j + const.,
\label{eq:ising_h}
\end{equation}
$$

and our QUBO problem would be equivalent to finding a state $\ket{\psi} = \sum_x c_x \ket{x}$ that minimizes the expectation value of our _Ising Hamiltonian_.: 

$$
\begin{equation*}
\expval{\hat{H}_C} = \frac{\bra{\psi}\hat{H}_C\ket{\psi}}{\braket{\psi}{\psi}}.
\end{equation*}
$$

In order to be able to compare the average energy levels of arbitrary states, we should use the expectation value instead of eigenvalue; firstly, because the arbitrary state might not be an eigenstate of our Hamiltonian and secondly, it might not be normalized. 

It is straightforward to check that the state that minimizes the above expectation value is the ground state of our system. However, finding the minimal energy of complicated Hamiltonians are analytically impossible and one should tap into _perturbative calculations_ and other approximation methodologies to attack this problem.


## Variational Quantum Eigensolver (VQE)

Consider the Hamiltonian $\hat{H}_C$ and $E_0$ as the ground state energy of this Hamiltonian. It is worth recapping the seemingly trivial but extremely powerful _variational principle of quantum mechanics_. 


> __Variational Principle of Quantum Mechanics__
>
>For any arbitrary _trial state_ $\ket{\psi}$, the average energy level of the state is greater than or equal to $E_0$:
>
>$$
\begin{equation*}
\expval{\hat{H}_C}_{\psi} \ge E_0
\end{equation*}
$$


The objective is to start from an initial trial state, evaluate its energy, and step-by-step modify the state in a way that takes us closer and closer to the ground state. But how should we modify the trial states to get closer to the optimal solution (the ground state)? We should do unitary state modifications in the corresponding Hilbert space (another name for doing rotations and inversions in the Hilbert space); however, a guideline is needed to tell us the direction and amount of change at each step. 

<a id='ansatz'></a>
### Parameterization (ansatz circuit or variational from)

The required guideline could be formulated on the basis of a set of optimization parameters $\theta$:
- Trial states need to be parameterized. This means that the trial state now becomes $\ket{\psi(\theta)}$ and it would be modified on the basis of the changes we make in $\theta$.

$$
\begin{equation*}
\ket{\psi(\theta)} = \hat{U}(\theta)\ket{0}
\end{equation*}
$$

- _Parameterized_ quantum circuits (quantum operators) would be required as agents of change. These parameterized circuits would prepare $\ket{\psi(\theta)}$ at the beginning of each iteration and evaluate $\expval{\hat{H}_C}_{\psi(\theta)}$.

> __Generic Near-Term Parameterization__
>
> Parameterization for near-term quantum computers is mainly achieved starting by fixed/parameterized single-qubit rotations on all qubits at the entrance of the circuit followed by fixed/parameterized two-qubit entangling gates. The final structure could have layers, blocks, or trees.


<a id='algorithm'></a>
### Algorithm

Variational Quantum Eigensolvers are a set of hybrid quantum-classical algorithms specifically developed for near-term quantum computers. The cornerstone is parametrization :

0) parameterize trial states and the corresponding quantum circuit; objective is to find the optimal set of parameters:

$$
\begin{equation*}
\theta_{opt} = {\argmin}_{\theta} \expval{\hat{H}_C}_{\psi(\theta)}
\end{equation*}
$$

> __Parameterizarion Quality__ 
>
> The exact method of parametrization mainly depends on quantum hardware considerations and optimization performance factors. In the end, the quality of the result would depend on this parametrization.

At iteration $n$ of the algorithm do the following:

1) __evaluate energy__ $\to$ setup the quantum circuit with $\theta_n$ and evaluate the energy of the trial state $\ket{\psi(\theta_n)}$ using the circuit.


2) __optimize $\theta$__ $\to$ on a classical computer, run a classical optimization algorithm such as _gradient descent_, _natural gradient_, _sampling gradient_ and so forth. Generally, these algorithms would take into account the previous iterations results to update $\theta$, i.e. to find $\theta_{n+1}$ for the next iteration quantum circuit setup.


## Example

As an example, we will implement a VQE for solving a MaxCut using QisKit predefined [`EfficientSU2`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.EfficientSU2.html#qiskit.circuit.library.EfficientSU2) as the ansatz ([N-local circuit](https://qiskit.org/documentation/apidoc/circuit_library.html)) and [`GSLS`](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.GSLS.html) as the classical [optimizer](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.html).

You could run the following cell to install the requirements.

In [ ]:
# install qiskit_optimization if ModuleNotFound

import sys
import subprocess

try:
    import qiskit_optimization
except ModuleNotFoundError:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', 'qiskit_optimization'], stdout=subprocess.DEVNULL)

### Generate random undirected graph

In [ ]:
import networkx as nx
import numpy as np
import random
#
# generate random undirected graph with 
# n-verices and m-edges
#
n = 5 
m = 7 
g = nx.gnm_random_graph(n, m)
for (u, v, w) in g.edges(data=True):
    w['weight'] = random.randint(1, 5)
#
# print edges data
#
c = 0
for edge in g.edges.data():
    c += 1
    print("edge %d : connecting %d to %d - %s" % (c, edge[0], edge[1], edge[2]))
#
# construct the weight matrix
#
w = np.zeros([n, n])
for i in range(n):
    for j in range(n):
        temp = g.get_edge_data(i, j, default=0)
        if temp != 0:
            w[i, j] = temp["weight"]
print('-'*60)
print('weight matrix : ')
print(w)
print('-'*60)

### Find the Ising Hamiltonian

The first step in any VQE is to find the Ising Hamiltonian analogous to the QUBO at hand. 

In the code below, the object `qp` stores the raw Quadratic Binary expression (cost function) to be maximized. Consequently, taking advantage of the variable change in Eq.$~\ref{eq:var_change}$, the Ising Hamiltonian analogous to our QUBO expression would be constructed. This is done using the method `to_ising()` whose output would be the Ising Hamiltonian in Eq.$~\ref{eq:ising_h}$, i.e. there would only be $\frac{1}{2}w_{ij}\hat{Z}_i\hat{Z}_j$ terms (the object `hamiltonian`) and a constant term (the object `offset`) whose role is to shift all energy eigenvalues by a single value. 

It is noteworthy that no single-qubit inversion term is present in the Ising model for QUBO problems. In the case of manipulating spins, such terms come from the interaction of a spin-1/2 particle with an external magnetic field while $\hat{Z}_i\hat{Z}_j$ terms are related to magnetic interactions between particle $i$ and particle $j$.

In [ ]:
from qiskit_optimization.applications import Maxcut

maxcut = Maxcut(g)
qp = maxcut.to_quadratic_program()
print(qp.prettyprint())

hamiltonian, offset = qp.to_ising()
#
# the coeffs of Z_i*Z_j in the Ising Hamiltonian would be 
# a quarter of the corresponding coeffs in qp.
#
print('The corresponding Ising Hamiltonian: \n',hamiltonian,'\n+ (',offset,')')

<div style="border: 2px solid green; padding: 10px;">

To clarify the output of `to_ising()`, several remarks should be made:

- To simplify things, the terms $x_i x_j$ are re-arranged with the constraint $i < j$ in the QUBO cost function; this is allowed as $x_i$'s and $x_j$'s commute and is also explicitly mentioned in Eq.$~\ref{eq:ising_h}$.
- The order of the qubits in the ouput of `to_ising()` are reversed; for instance, a term `2.0 * IZIIZ` in the output is equivalent to $2 \hat{Z}_0\hat{Z}_3$.
- As mentioned earlier, the variable change would make terms like $c_i \hat{Z}_i$ cancel out. In fact, the `offset` is simply $-\frac{1}{4}\sum_i d_i$ where $d_i$ is the coefficient of $x_i$ in the QUBO cost function.

</div>

### Prepare VQE

We are going to use `EfficientSU2` for parameterization; however, any customized ansatz could have been defined and used instead. And `GSLS` will be used as the classical parameter optimizer.

#### ansatz circuit
   
As discussed in [VQE - Algorithm](#algorithm), the first step in a VQE deployment is _parameterization_. 

The outcome of parametrization is a quantum circuit that executes the quantum part of the VQE. Parameterization is defining/selecting your [ansatz circuit](#ansatz).

`EfficientSU2` is a circuit scheme initiated by a layer of two consecutive parameterized sets of $SU(2)$ rotations (by default $R_y$ followed by $R_z$ performed on all qubits). Afterwards, a series of parametrized entangling operations are performed (composed of $SU(2)$ rotations and CNOT). These parameterized entanglements simulate approximations of the interaction terms in the Ising Hamiltonian. Therefore, the exact configuration of the enangling layer depends on which interaction terms are present in the Ising Hamiltonian of the problem. 

Clearly, the number of paramters to be optimized is one of the specifications of the ansatz circuit.

#### paramter optimization

In the case of a 5-qubit `EfficientSU2` ansatz circuit, the number of paramaters to be optimized is 27! At the end of each iteration, the classical optimizer should decide how to modify every single one of these parameters for the next run on the ansatz circuit. 

As mentioned before, `GSLS` (Gaussian-Smoothed Line Search) has been selected as the classical optimizer of our VQE. The mission of the classical optimizer is to determine the parameters of the ansatz circuit for its next run. In other words, it would specify _step size and descent direction_ for the next iteration. 

The way `GSLS` selects our steps for each iteration is by _smoothing_ the function over our dataset (energy values as a function of the parameters) using a Gaussian kernel and then for each parameter, it would try to find the next value which could minimize the Gaussian-smoothed fuction (this is called a _line search algorithm_).

Smoothing is a method of finding some sort of patterns within a dataset by ignoring small fluctuations. Therefore, after smoorhing, our dataset has indeed been modified. In the case of `GSLS`, smoothing is done by taking points, one-by-one, in our dataset and replacing each energy value by a weighted average of the energey values of the neighboring points. The weights used in the averaging follows a Gaussian profile; i.e. points with less distance to our considered point have higher weights. It is not hard to see that the standard deviation of the selected Gaussian kernel would determine the amount of smoothing to be done.

In [ ]:
#
# use EfficientSU2 as the ansatz circuit
#
from qiskit.circuit.library.n_local import EfficientSU2
#
# deploy Estimator to evaluate expectation values
#
from qiskit.primitives import Estimator
#
# deploy GSLS as the classical local optimizer
#
from qiskit.algorithms.optimizers import GSLS
#
# deploy VQE as the variational algorithm
#
from qiskit.algorithms.minimum_eigensolvers import VQE

ansatz = EfficientSU2(num_qubits=n, reps=2)

vqe = VQE(estimator=Estimator(), ansatz=ansatz, optimizer=GSLS())

print(ansatz.decompose())

#ansatz.decompose().draw("mpl", style="iqx")

### Run VQE for the problem's Hamiltonian

This part might take some time depending on the number of parameters used by the ansatz circuit.

In [ ]:
result = vqe.compute_minimum_eigenvalue(hamiltonian)

In [ ]:
print('-'*60)
print('iterations : ', result.cost_function_evals)
print('optimizer time : ', result.optimizer_time)
print('found optimal energy : ', result.optimal_value.real + offset)
print('-'*60)

The object `result` embodies circuit `optimal_parameters` and the _energy_ of the optimal trial state (`optimal_value`). In order to find the optimal trial state, we should do _sampling_. This could have been done by deploying `SamplingVQE` instead of `VQE` which would do an automatic sampling when optimal parameters are obtained. Now that we have the optimal circuit parameters, we could perform the sampling on the circuit ourselves.

In [ ]:
from qiskit.primitives import Sampler

optimal_state = ansatz.bind_parameters(result.optimal_parameters)
optimal_state.measure_all()

sampler = Sampler()
distribution = sampler.run([optimal_state]).result().quasi_dists[0]

solution = max(distribution.binary_probabilities().items(), key=lambda x: x[1])

for entry in distribution.binary_probabilities().items():
    print("state : %s , probability : %f" % (entry[0], entry[1]))

print('-'*60)
print('Optimal State : ', solution[0])
print('Probability : ', solution[1])
print('-'*60)

### Wrap up

In this example, we used a predefined Two-Local circuit as the ansatz. It could be a useful exercise to checkout the Ising Hamiltonian for your problem, define your own ansatz circuit, and compare obtained results for different ansatz. In comparing different ansatz, the impact of the classical optimizer should not be underestimated! An optimizer should meaningfully suit the ansatz.